In [1]:
import torch
import torch.utils.data
import matplotlib.pyplot as plt

import yaml

from configs.trainer import TrainingConfig

from models.vision_encoder_decoder import VisionEncoderDecoder

from transformers import AutoTokenizer, PreTrainedTokenizer
from deeplake import load, Dataset
from torchvision.models import ViT_B_16_Weights

/Users/dinesh/miniforge3/envs/image_captioning/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_str" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/dinesh/miniforge3/envs/image_captioning/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [3]:
def get_dataloader(batch_size, shuffle):
    ds: Dataset = load('hub://activeloop/flickr30k')
    val_dl = ds.query("SELECT * WHERE ROW_NUMBER() >= 27000 "). \
        pytorch(batch_size=batch_size, shuffle=shuffle, num_workers=0,
                buffer_size=32, use_local_cache=True)
    return val_dl


In [4]:
config_file = 'training_configs/local/nano.yaml'
chkpt_file = 'checkpoints/nano.pt'

In [5]:
obj = yaml.safe_load(open(config_file, 'r'))
config: TrainingConfig = TrainingConfig.parse_obj(obj)
config.model.chkpt_path = chkpt_file
print(config)

model=VisionEncoderDecoderConfig(vision_encoder_config=ViTConfig(refine_base_model=False, n_embd_out_vit=768, n_cls=16, gate_sizes=(1024,)), decoder_config=TransformerDecoderConfig(use_advanced_pos_emb=False, advanced_pos_emb_gate_sizes=None, pretrained_model=<ModelType.GPT2: 'gpt2'>, enable_gradient_checkpointing=False, n_layer=12, skip_alternate_cross_attn=True, block_size=1024, vocab_size=50257, transformer_config=TransformerConfig(rotator_config=MLPConfig(ff_mult=4.0), is_causal=False, is_cross_attn=True, max_block_size=None, is_sparse_attn=False, sparsity_factor=0.5, attn_config=SelfAttentionConfig(attn_dropout=0.1, bias=True, dropout=0.1, n_head=12, n_embd=768, attn_type=<SelfAttentionType.MULTI_HEAD: 'multi_head'>))), loose_match_decoder_state_dict=True, chkpt_path='checkpoints/nano.pt', use_cross_attn=True, use_soft_prompting=True, no_repeat_n_grams=(2, 3, 4, 5)) disable_flash=False ignore_index=-100 batch_size=4 dataloader_buffer_size=5 shuffle=True gradient_accumulation_steps

/var/folders/1n/tqgq4c0j5p7cs5dk9n284h7c0000gn/T/ipykernel_2959/3115359875.py:2: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.0.3/migration/
  config: TrainingConfig = TrainingConfig.parse_obj(obj)


In [6]:
tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(config.tokenizer_str)
kwargs = {}
if tokenizer.eos_token_id is None:
    kwargs['eos_token'] = '<EOS>'
if tokenizer.bos_token_id is None:
    kwargs['bos_token'] = '<BOS>'
if tokenizer.mask_token_id is None and config.trainer.mask_fraction > 0:
    kwargs['mask_token'] = '<MSK>'
tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(
    config.tokenizer_str, **kwargs)


'<|endoftext|>'

In [ ]:
val_dl = get_dataloader(1, False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.has_mps else 'cpu')

In [ ]:
model = VisionEncoderDecoder(config.model).to(device)


In [ ]:
model.eval()

In [ ]:
# num_beams = 4
# num_new_tokens = 64
# top_k = 16
# temperature = 1.0
# consolidation_temperature = 100.0
# batch_size = 1
ignore_index = -100
# beam_expansion_factor = 8
# length_boost = 1.0

num_candidates = 8

In [ ]:
# generator = BeamSearchTokenGenerator(
#     model,
#     beam_width=num_beams,
#     temperature=temperature,
#     consolidation_temperature=consolidation_temperature,
#     max_new_tokens=num_new_tokens,
#     no_repeat_n_grams=(2, 3, 4, 5),
#     top_k=top_k,
#     beam_expansion_factor=beam_expansion_factor,
#     eos_token_id=tokenizer.eos_token_id,
#     length_boost=length_boost,
# )

In [ ]:
tx = ViT_B_16_Weights.IMAGENET1K_SWAG_LINEAR_V1.transforms()


for i, batch in enumerate(val_dl):
    if i == 20:
        break

    x = torch.tensor(batch['image'])
    plt.imshow(x[0])
    plt.show()

    x = tx(x[0].permute(2, 0, 1)).unsqueeze(0)

    prompt = tokenizer.bos_token
    prompt_ids = torch.tensor(tokenizer(text=prompt).input_ids, dtype=torch.long).to(x.device).unsqueeze(0).expand(x.size(0), -1).contiguous()
    x = x.to(device).expand(num_candidates, -1, -1, -1)

    decoded_ids = torch.tensor(
        tokenizer(text=prompt).input_ids,
        dtype=torch.long).to(device).unsqueeze(0).expand(x.size(0), -1).contiguous()

    result = model.generate(images=x,
                            prompt_ids=decoded_ids,
                            temperature=1.0,
                            max_new_tokens=64,
                            top_k=16)
    result = tokenizer.batch_decode(result)

    print('truth', batch['caption_0'][0], batch['caption_1'][0], batch['caption_2'][0], batch['caption_3'][0], batch['caption_4'][0], '\n')
    for gen in result:
        i = gen.find(tokenizer.eos_token)
        gen = gen[:i]
        print(gen)
    print("========================================================")